# Load data

In [21]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/gestures.csv')
print(df.head())


#separate labels
X = df.iloc[:, 1:].values  # all columns except the label
y = df.iloc[:, 0].values   # label column

  label        x0        y0            z0        x1        y1        z1  \
0  open  0.208197  0.932596  6.621586e-07  0.276534  0.870955 -0.023450   
1  open  0.207864  0.930138  6.584116e-07  0.276500  0.870342 -0.023726   
2  open  0.207823  0.931254  6.625651e-07  0.276393  0.871789 -0.024244   
3  open  0.209642  0.932560  6.785308e-07  0.279597  0.871208 -0.023636   
4  open  0.210168  0.934793  6.746104e-07  0.280106  0.873031 -0.023108   

         x2        y2        z2  ...       z17       x18       y18       z18  \
0  0.327159  0.789608 -0.033913  ... -0.028019  0.097139  0.641211 -0.041787   
1  0.327573  0.789807 -0.034169  ... -0.027804  0.097646  0.640679 -0.041928   
2  0.329070  0.791110 -0.034874  ... -0.027773  0.098512  0.641017 -0.042177   
3  0.331479  0.791676 -0.033870  ... -0.025761  0.100507  0.641706 -0.039729   
4  0.331285  0.791756 -0.032947  ... -0.025479  0.100386  0.640941 -0.039158   

        x19       y19       z19       x20       y20       z20  
0  0

# Normalize landmarks

In [9]:
def normalize_landmarks(landmarks):
    landmarks = landmarks.reshape(-1, 3)
    wrist = landmarks[0]                      # landmark 0 = wrist
    landmarks -= wrist                        # translate
    scale = np.linalg.norm(landmarks[9] - wrist)  # distance to middle finger base
    landmarks /= scale
    return landmarks.flatten()

X_norm = np.array([normalize_landmarks(x) for x in X.reshape(len(X), 21, 3)])

# Encode gestures

In [10]:
# !pip install tensorflow
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = to_categorical(y_encoded)

print(le.classes_)  # ['fist', 'open', 'peace']

['fist' 'open' 'peace']


# Split the data

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(
    X_norm, y_onehot, test_size=0.3, stratify=y_encoded, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp.argmax(axis=1), random_state=42
)

print(len(X_train), len(X_val), len(X_test))

847 182 182


In [12]:
import os
print("Current working directory:", os.getcwd())

Current working directory: c:\AUsers\Amalia\Facultate\An4\Block1\Ai\Computer_Vision\notebooks


In [18]:
import numpy as np

np.savez_compressed('../data/processed_gestures.npz',
                    X_train=X_train, y_train=y_train,
                    X_val=X_val, y_val=y_val,
                    X_test=X_test, y_test=y_test)